In [ ]:
#!pip install sklearn -q
#!pip install spacy -q
#!python -m spacy download en_core_web_sm
#!pip install gensim

In [ ]:
import numpy as np
import pandas as pd
import string
import spacy
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

#import nltk
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api
from gensim.models import KeyedVectors

from google.colab import drive
drive.mount('/content/drive/')

np.random.seed(42)

[nltk_data] Downloading package wordnet to /root/nltk_data...


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/CodSoft Datasets/spam.csv", encoding='ISO 8859-1')
df = df.astype(str)
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",nan,nan,nan
1,ham,Ok lar... Joking wif u oni...,nan,nan,nan
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,nan,nan,nan
3,ham,U dun say so early hor... U c already then say...,nan,nan,nan
4,ham,"Nah I don't think he goes to usf, he lives aro...",nan,nan,nan


In [ ]:
print(list(gensim.downloader.info()['models'].keys()))
wv = api.load('word2vec-google-news-300')
wv.save('/content/drive/MyDrive/CodSoft Datasets/google-300-vectors.kv')

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']
[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
#wv = KeyedVectors.load('/content/drive/MyDrive/CodSoft Datasets/wiki-100-vectors.kv')
#wv = KeyedVectors.load('/content/drive/MyDrive/CodSoft Datasets/google-300-vectors.kv')

In [ ]:
# Combine the dataset in a way that there are no unnecessary columns
def combine_except_first(row):
    return ' '.join(row[1:])

df['combined_message'] = df.apply(combine_except_first, axis=1)
df = df[['v1', 'combined_message']]
df['clean_message'] = df['combined_message']
df.head()

<ipython-input-79-931721ca5e88>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_message'] = df['combined_message']


,v1,combined_message,clean_message
0,ham,"Go until jurong point, crazy.. Available only ...","Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni... nan nan nan,Ok lar... Joking wif u oni... nan nan nan
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro...","Nah I don't think he goes to usf, he lives aro..."


In [ ]:
count_dict = dict()

for row in df['combined_message']:
    wrd = row.split()

    for alp in wrd:
        if alp in count_dict:
            count_dict[alp] += 1
        else:
            count_dict[alp] = 1

count_list = list(count_dict.items())

sorted_word_count_list = sorted(count_list, key=lambda x: x[1], reverse=True)
print(sorted_word_count_list)

[('nan', 16649), ('to', 2145), ('you', 1626), ('I', 1468), ('a', 1336), ('the', 1207), ('and', 858), ('in', 800), ('is', 787), ('i', 749), ('u', 699), ('for', 650), ('my', 630), ('of', 591), ('me', 561), ('your', 560), ('on', 488), ('have', 477), ('2', 457), ('that', 422), ('it', 397), ('are', 397), ('call', 376), ('or', 373), ('be', 363), ('at', 360), ('with', 351), ('not', 344), ('will', 333), ('get', 326), ('can', 304), ('U', 301), ('ur', 297), ('so', 295), ("I'm", 286), ('but', 286), ('&lt;#&gt;', 276), ('You', 263), ('from', 256), ('4', 255), ('do', 243), ('up', 242), ('.', 241), ('if', 239), ('just', 239), ('go', 235), ('when', 234), ('this', 225), ('like', 224), ('know', 224), ('we', 222), ('all', 218), ('out', 207), ('got', 204), ('was', 200), ('come', 199), ('now', 190), ('?', 187), ('am', 186), ('...', 163), ('want', 158), ('by', 156), ('Call', 155), ('time', 154), ('about', 150), ('send', 150), ('only', 147), ('then', 146), ('what', 143), ('going', 142), ('need', 142), ('n',

In [ ]:
# Clean the dataset by making all alphabets lower case, removing left and right spaces, replacing new line character with space and removing standalone letters
def clean_text(row):
  row = str(row).lower()
  row = row.lstrip()
  row = row.rstrip()
  row = re.sub('\n', '', row)
  row = re.sub('\w*\d\w*', '', row)
  row = re.sub(r'\.', '', row)
  row = re.sub('[%s]' % re.escape(string.punctuation), '', row)
  return row

In [ ]:
# Removing all unnecessary stop words that are common and do not contribute towards the model
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
stop_words.add('nan')
stop_words.add('u')

def remove_stopwords(row):
    row = ' '.join(word for word in row.split(' ') if word not in stop_words)
    return row

In [ ]:
# Applying lematization on the dataset
# Since the dataset is focused on finding spam messages, the meaning is more important hence lematization

lt = WordNetLemmatizer()
def lematizer(row):
    row = ' '.join(lt.lemmatize(word) for word in row.split(' '))
    return row

In [ ]:
#Applying all Data Cleaning Functions onto the Dataset

def cleanData():
    df['clean_message'] = df['clean_message'].apply(clean_text)
    df['clean_message'] = df['clean_message'].apply(remove_stopwords)
    df['clean_message'] = df['clean_message'].apply(lematizer)

cleanData()
df.head(10)

,v1,combined_message,clean_message
0,ham,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis n great wor...
1,ham,Ok lar... Joking wif u oni... nan nan nan,ok lar joking wif oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry wkly comp win fa cup final tkts ...
3,ham,U dun say so early hor... U c already then say...,dun early hor c
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think go usf life
5,spam,FreeMsg Hey there darling it's been 3 week's n...,freemsg hey darling week word id like fun tb ...
6,ham,Even my brother is not like to speak with me. ...,brother like speak treat like aid patent
7,ham,As per your request 'Melle Melle (Oru Minnamin...,request melle melle oru minnaminunginte nurung...
8,spam,WINNER!! As a valued network customer you have...,winner valued network customer selected receiv...
9,spam,Had your mobile 11 months or more? U R entitle...,mobile month r entitled update latest colour ...


In [ ]:
def tokenizer(row):
    return row.split()

df['tokens'] = df['clean_message'].apply(tokenizer)
df.head()

,v1,combined_message,clean_message,tokens
0,ham,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis n great wor...,"[jurong, point, crazy, available, bugis, n, gr..."
1,ham,Ok lar... Joking wif u oni... nan nan nan,ok lar joking wif oni,"[ok, lar, joking, wif, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry wkly comp win fa cup final tkts ...,"[free, entry, wkly, comp, win, fa, cup, final,..."
3,ham,U dun say so early hor... U c already then say...,dun early hor c,"[dun, early, hor, c]"
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think go usf life,"[nah, dont, think, go, usf, life]"


In [ ]:
def sent_vec(sent):
    vector_size = wv.vector_size
    wv_res = np.zeros(vector_size)
    # print(wv_res)
    ctr = 1
    for w in sent:
        if w in wv:
            ctr += 1
            wv_res += wv[w]
    wv_res = wv_res/ctr
    return wv_res

In [ ]:
df['vec'] = df['tokens'].apply(sent_vec)
df.head()

,v1,combined_message,clean_message,tokens,vec
0,ham,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis n great wor...,"[jurong, point, crazy, available, bugis, n, gr...","[-0.017922537667410716, 0.046787806919642856, ..."
1,ham,Ok lar... Joking wif u oni... nan nan nan,ok lar joking wif oni,"[ok, lar, joking, wif, oni]","[-0.020917256673177082, 0.07261149088541667, 0..."
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry wkly comp win fa cup final tkts ...,"[free, entry, wkly, comp, win, fa, cup, final,...","[-0.026138754451976103, -0.014562270220588236,..."
3,ham,U dun say so early hor... U c already then say...,dun early hor c,"[dun, early, hor, c]","[-0.00078125, 0.07607421875, 0.0714111328125, ..."
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think go usf life,"[nah, dont, think, go, usf, life]","[-0.028773716517857144, 0.039031982421875, 0.0..."


In [ ]:
df.loc[df['v1'] == 'ham', 'v1'] = 0
df.loc[df['v1'] == 'spam', 'v1'] = 1
df.head(10)

,v1,combined_message,clean_message,tokens,vec
0,0,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis n great wor...,"[jurong, point, crazy, available, bugis, n, gr...","[-0.017922537667410716, 0.046787806919642856, ..."
1,0,Ok lar... Joking wif u oni... nan nan nan,ok lar joking wif oni,"[ok, lar, joking, wif, oni]","[-0.020917256673177082, 0.07261149088541667, 0..."
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entry wkly comp win fa cup final tkts ...,"[free, entry, wkly, comp, win, fa, cup, final,...","[-0.026138754451976103, -0.014562270220588236,..."
3,0,U dun say so early hor... U c already then say...,dun early hor c,"[dun, early, hor, c]","[-0.00078125, 0.07607421875, 0.0714111328125, ..."
4,0,"Nah I don't think he goes to usf, he lives aro...",nah dont think go usf life,"[nah, dont, think, go, usf, life]","[-0.028773716517857144, 0.039031982421875, 0.0..."
5,1,FreeMsg Hey there darling it's been 3 week's n...,freemsg hey darling week word id like fun tb ...,"[freemsg, hey, darling, week, word, id, like, ...","[-0.006649310772235577, 0.02625450721153846, 0..."
6,0,Even my brother is not like to speak with me. ...,brother like speak treat like aid patent,"[brother, like, speak, treat, like, aid, patent]","[0.0092010498046875, 0.073211669921875, 0.1031..."
7,0,As per your request 'Melle Melle (Oru Minnamin...,request melle melle oru minnaminunginte nurung...,"[request, melle, melle, oru, minnaminunginte, ...","[0.04046630859375, 0.0521697998046875, 0.01696..."
8,1,WINNER!! As a valued network customer you have...,winner valued network customer selected receiv...,"[winner, valued, network, customer, selected, ...","[-0.039663461538461536, -0.015432504507211538,..."
9,1,Had your mobile 11 months or more? U R entitle...,mobile month r entitled update latest colour ...,"[mobile, month, r, entitled, update, latest, c...","[0.014700753348214286, -0.09211077008928571, -..."


In [ ]:
X = df['vec'].to_list()
y = df['v1'].to_list()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y)

In [ ]:
classifier = LogisticRegression()
classifier.fit(X_train,y_train)

LogisticRegression()

In [ ]:
predicted = classifier.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.9614349775784753
Logistic Regression Precision: 0.8955223880597015
Logistic Regression Recall: 0.8053691275167785
